<a href="https://colab.research.google.com/github/marinaotokk/banco-midias/blob/main/caixeiroviajante_gen.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Algortimos Genéticos**

* São técnicas de Computação Evolucionária centrada na busca de soluções para problemas de otimização.

## **Problema do Caixeiro Viajante**

O probelma possui o objetivo de encontrar a menor rota possível para visitar um conjunto de cidades, passando por cada uma delas uma única vez, e retornar à origem.


## **Biblioteca DAEP**
Para o desenvolvimento do problema foi utilizada a biblioteca DEAP https://deap.readthedocs.io/en/master/ que é nova estrutura de computação evolutiva para desenvolvimento de algoritmos genéticos.

In [1]:
# 1. Pacotes e módulos

import random
import numpy


from deap import algorithms
from deap import base
from deap import creator
from deap import tools

ModuleNotFoundError: No module named 'deap'

In [ ]:
# 2. Grafos para o problema

# função grafoPCV(numberCities, minDist, maxDist)
# parâmetros:
#   numberCities: número de cidades
#   minDist: menor distância
#   maxDist: maior distância
# return:
#   cidades: grafo de cidades (Matriz numCities X numCities).
#   Distâmcoa entre duas cidades são determinadas aleatoriamente entre minDist e maxDist
def grafoPCV(numberCities, minDist, maxDist):
  cities = numpy.zeros((numberCities, numberCities), dtype = int)
  for i in range(numberCities):
    for j in range(numberCities):
      if (j>i):
        cities[i, j] = random.randint(minDist, maxDist)
      elif (j<i):
        cities[i, j] = cities[j, i]
  return cities

numberCities = 5     #  Número de cidade inicial

while(True):
  numberCities = int(input('Digite o número de cidades: '))
  if (numberCities > 4):
    break
  else:
    print('O número de cidades deve ser maior que 4!')

cities = grafoPCV(numberCities, 10, 100)
print('Grafo:\n', cities)

In [ ]:
# 3. Definir Geração dos Indivíduos

creator.create("FitnessMin", base.Fitness, weights=(-1.0,)) # peso negativo para solução miníma
creator.create("Individual", list, fitness=creator.FitnessMin)

toolbox = base.Toolbox()
# gerador de parâmetros
toolbox.register("attr_int", random.randint, 0, numberCities-1)

# definir como os indivíduos/população são gerados
toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_int, numberCities)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

## **Avaliação de Aptidão**
A Função de Aptidão ou Fitness mede o grau de aptidão de cada indivíduo da população.

> No caso do **Problema do Caixeiro Viajante** a aptidão está associada a menor rota.

In [ ]:
# 4. Função para avaliar a aptidão

# Função evalRoute(individual)
# parâmetros:
#   individual: uma rota
# retorno:
#   (cost, ): tupla contendo apenas o custo da rota (cost). * é uma tupla por causa das exigências do pacote DEAP
def evalRoute(individual):
  cost = 0
  for i in range(1, len(individual)):
    if (individual.count(individual[i])>1):
      cost = cost + 1000000 # penalidade por repetição da cidade
    cost = cost + cities[individual[i-1], individual[i]]
  cost = cost + cities[individual[i],individual[0]]
  return (cost,)

## **Processamento do Algoritmo Genético**


In [ ]:
#Plota o grafo com a melhor rota encontrada
def plot_graph(cities, route=None):
    G = nx.Graph()
    for i in range(len(cities)):
        for j in range(i+1, len(cities)):
            G.add_edge(i, j, weight=cities[i][j])

    pos = nx.spring_layout(G)
    nx.draw(G, pos, with_labels=True, node_color='skyblue', node_size=700)
    labels = nx.get_edge_attributes(G, 'weight')
    nx.draw_networkx_edge_labels(G, pos, edge_labels=labels)

    if route:
        edges = [(route[i-1], route[i]) for i in range(len(route))]
        G_route = nx.Graph()
        G_route.add_edges_from(edges)
        nx.draw(G_route, pos, edge_color='red', width=2)

    plt.show()

#5. Processamento do Algoritmo Genético

# Definindo avaliação de aptidão, seleção, cruzamento e mutação
toolbox.register("evaluate", evalRoute)
toolbox.register("select", tools.selTournament, tournsize=3) # seleção por torneio
toolbox.register("mate", tools.cxOnePoint) # um ponto de cruzamento
toolbox.register("mutate", tools.mutUniformInt, low=0, up=numberCities-1, indpb=0.05)

def main():
  print('Execução do algoritmo genético:')

  # random.seed(64)
  NGEN = 100     # número de gerações
  MU = 50        # tamanho da população
  LAMBDA = 100   # número de filhos gerados
  CXPB = 0.7     # probabilidade de cruzamento
  MUTPB = 0.3    # probabilidade de mutação

  pop = toolbox.population(n=MU)
  hof = tools.ParetoFront()
  stats = tools.Statistics(lambda ind: ind.fitness.values)
  stats.register("avg", numpy.mean, axis=0)
  stats.register("std", numpy.std, axis=0)
  stats.register("min", numpy.min, axis=0)
  stats.register("max", numpy.max, axis=0)

  algorithms.eaMuPlusLambda(pop, toolbox, MU, LAMBDA, CXPB, MUTPB, NGEN, stats,
                            halloffame=hof)

  print('\nRota:', hof[0],'\nCusto:', evalRoute(hof[0])[0])

  best_route = hof[0]
  plot_graph(cities, best_route)

  return pop, stats, hof

if __name__ == "__main__":
    main()